# BE 521 Final Project
## Group: The Chicken Finger Feature Finders
## Group Members: Carlos Aguila, Anthony LoPrete, Frederick Xu

Instructions:
1. Download "final_submission_files.zip" (~370 mb) from the following link: https://drive.google.com/file/d/1VmzfwPeyqEjlwJ6e_BF0SyCEyicAGLMJ/view?usp=sharing
2. Upload "truetest_data.mat" and "final_submission_files.zip" to the content directory.
3. Run the script, the notebook should do the rest.
4. The code outputs "predictions.mat", which contains a variable `predicted_dg`, which is our predictions on the hidden test set.

Some notes on "final_submission_files.zip":
- There are 3 `.model` files corresponding to a pre-trained algorithm for each subject
- There are 3 `.npy` files corresponding to the training features for each subject
- The 3 `.npy` files are *only for normalizing the hidden test set*. Our code was originally built to normalize all feature sets together in one function so these arrays are necessary for that code to work.

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.io import loadmat, savemat
from scipy.interpolate import CubicSpline
from scipy.signal import ellip, lfilter, filtfilt, find_peaks, butter, sosfiltfilt,sosfilt

In [1]:
!unzip "final_submission_files.zip" -d "."

Archive:  final_submission_files.zip
  inflating: ./all_feats_s1_train_fx.npy  
  inflating: ./all_feats_s2_train_fx.npy  
  inflating: ./all_feats_s3_train_fx.npy  
  inflating: ./subject1_rfr_1000_run2.model  
  inflating: ./subject2_rfr_1000_run2.model  
  inflating: ./subject3_rfr_1000_run2.model  


In [29]:
test_data = loadmat('truetest_data.mat')

#leaderboard ecog signal per patient
test_data_s1 = test_data[list(test_data.keys())[-1]][0][0]
test_data_s1 = np.delete(test_data_s1, 54, axis=1)

test_data_s2 = test_data[list(test_data.keys())[-1]][1][0]
test_data_s2 = np.delete(test_data_s2, [20, 37], axis=1)

test_data_s3 = test_data[list(test_data.keys())[-1]][2][0]

## Signal pre-processing

We used an 8th-order Butterworth bandpass filter with a lower cutoff of 0.15 Hz and upper cutoff of 200 Hz. 

In [7]:
def filter_data(raw_eeg, fs=1000):
  """
  Write a filter function to clean underlying data.
  Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
  Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
  distorting the underlying data!

  Input: 
    raw_eeg (samples x channels): the raw signal
    fs: the sampling rate (1000 for this dataset)
  Output: 
    clean_data (samples x channels): the filtered signal
  """
  raw_eeg_t = raw_eeg.transpose()
  filtered = []

  nyq = fs/2

  sos = butter(8, [0.15,200],btype='bandpass',output='sos',fs=fs)

  for ch_data in raw_eeg_t:
    filtered_ch = sosfiltfilt(sos,ch_data)
    filtered.append(filtered_ch)
  
  filtered = np.array(filtered)

  return filtered.transpose()

In [8]:
filt_test_ecog_s1 = filter_data(test_data_s1)
filt_test_ecog_s2 = filter_data(test_data_s2)
filt_test_ecog_s3 = filter_data(test_data_s3)

## Feature Extraction

We extract several different features:
- Line Length
- Energy
- Biorhythm bandpower (Delta, Theta, Alpha, Beta, Gamma)

Please note: this step does take some time to run (~10 mins) to extract all of the features.

In [9]:
#number of windows in signal given winLen and winDisp
def NumWins(x, fs, winLen, winDisp):
  return (len(x) - winLen*fs + winDisp*fs) // (winDisp * fs)

#line length
def LL(x):
  return np.sum(np.absolute(np.ediff1d(x)))

#energy
def E(x):
  return np.sum(x**2)

# Bandpower, specifying the following:
# Delta: fmin = 0.5, fmax = 4
# Theta: fmin = 4, fmax = 7
# Alpha: fmin = 8, fmax = 12
# Beta: fmin = 12.5, fmax = 30
# Gamma: fmin = 25, fmax = 140
def bandpower(x, fs, fmin, fmax):
    f, Pxx = scipy.signal.periodogram(x, fs=fs)
    ind_min = np.argmax(f > fmin) - 1
    ind_max = np.argmax(f > fmax) - 1
    return np.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max])

#gets features, load features you want calculated from here
def get_features(filtered_window, fs=1000):
  """
    Write a function that calculates features for a given filtered window. 
    Feel free to use features you have seen before in this class, features that
    have been used in the literature, or design your own!

    Input: 
      filtered_window (window_samples x channels): the window of the filtered ecog signal 
      fs: sampling rate
    Output:
      features (channels x num_features): the features calculated on each channel for the window
  """

  filtered_window_t = filtered_window.transpose()

  features = []

  for ch in filtered_window_t:
    #features.append(np.array([LL(ch), E(ch), SP_5(ch), SP_20(ch), SP_75(ch), SP_125(ch), SP_160(ch)]))
    
    features.append(np.array([LL(ch), # Line-Length
                              E(ch), # Energy
                              bandpower(ch, fs, 0.5, 4), # Delta
                              bandpower(ch, fs, 4, 7), # Theta
                              bandpower(ch, fs, 8, 12), # Alpha
                              bandpower(ch, fs, 12.5, 30), # Beta
                              bandpower(ch, fs, 25, 140) # Gamma
                             ])) 
    
  features = np.array(features)

  return features

#get_windowed_feats - filters raw ecog signal and finds features
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
  """
    Write a function which processes data through the steps of filtering and
    feature calculation and returns features. Points will be awarded for completing
    each step appropriately (note that if one of the functions you call within this script
    returns a bad output, you won't be double penalized). Note that you will need
    to run the filter_data and get_features functions within this function. 

    Inputs:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)
      window_length: the window's length
      window_overlap: the window's overlap
    Output: 
      all_feats (num_windows x (channels x features)): the features for each channel for each time window
        note that this is a 2D array. 
  """

  cleaned_ecog = filter_data(raw_ecog)
  num_wins = NumWins(cleaned_ecog.transpose()[0], fs, window_length, window_overlap)
  all_feats_3d = []
  for winStart in np.arange(0, int(num_wins), 1):
    clip = cleaned_ecog[int(winStart*window_overlap*fs):int(winStart*window_overlap*fs + (window_length *fs))]
    all_feats_3d.append(get_features(clip))

  num_channels = len(all_feats_3d[0])
  num_features = len(all_feats_3d[0][0])

  all_feats = np.zeros([len(all_feats_3d),num_features*num_channels])

  for k in range(int(len(all_feats_3d))):
    q = flatten_list = [j for sub in all_feats_3d[k] for j in sub]
    all_feats[k,:] = q

  return np.array(all_feats)#, np.array(all_feats_3d)

In [14]:
all_feats_test_s1 = get_windowed_feats(filt_test_ecog_s1, 1000, 0.100, 0.050); #output of get_windowed_feats
all_feats_test_s2 = get_windowed_feats(filt_test_ecog_s2, 1000, 0.100, 0.050); #output of get_windowed_feats
all_feats_test_s3 = get_windowed_feats(filt_test_ecog_s3, 1000, 0.100, 0.050); #output of get_windowed_feats

In [15]:
print(all_feats_test_s1.shape)
print(all_feats_test_s2.shape)
print(all_feats_test_s3.shape)


(2949, 427)
(2949, 322)
(2949, 448)


## Normalizing the Features

We normalize the testing data based on the mean and standard deviation of the training data. For this, we have included the training data as part of the algorithm files and it is loaded accordingly after unzipping.

Please ignore the warnings, it is because some of the features ended up with all 0's resulting in a standard deviation of 0.

In [16]:
def normalize_features(all_feats_train, all_feats_test, num_features):
  # Input should be (num_windows x (channels x features))
  # Num features is the number of unique features that were extracted
  all_feats_train_norm = np.copy(all_feats_train)
  all_feats_test_norm = np.copy(all_feats_test)
  
  feats_avg = []
  feats_std = []
  
  for n in range(num_features):
    feats_idx_train = np.arange(n,len(all_feats_train.transpose()),num_features)
    
    feat_data_train = all_feats_train[:][:,feats_idx_train]
    
    feat_means_train = np.mean(feat_data_train,axis=0)
    feat_stds_train = np.std(feat_data_train,axis=0)
    
    all_feats_train_norm[:][:,feats_idx_train] = (feat_data_train - feat_means_train)/feat_stds_train
    
    # Note that we must use the same mean and std. dev from the TRAINING set
    #     because regression models are sensitive to value domain as they are
    #     scale-variant.
    feats_idx_test = np.arange(n,len(all_feats_test.transpose()),num_features)
    feat_data_test = all_feats_test[:][:,feats_idx_test]
    all_feats_test_norm[:][:,feats_idx_test] = (feat_data_test - feat_means_train)/feat_stds_train
    
    # Sanity checking plot, comment out if you don't want plots
    # if n == 0:
    #     plt.figure()
    #     plt.plot(feat_data_train.transpose()[0])
    #     plt.figure()
    #     plt.plot(all_feats_train_norm[:][:,feats_idx_train].transpose()[0])
    
    #     plt.figure()
    #     plt.plot(feat_data_test.transpose()[0])
    #     plt.figure()
    #     plt.plot(all_feats_test_norm[:][:,feats_idx_test].transpose()[0])

  return all_feats_train_norm, all_feats_test_norm

In [17]:
file_s1 = open("./all_feats_s1_train_fx.npy", "rb")
feats_s1_train = np.load(file_s1)
file_s1.close()

file_s2 = open("./all_feats_s2_train_fx.npy", "rb")
feats_s2_train = np.load(file_s2)
file_s2.close()

file_s3 = open("./all_feats_s3_train_fx.npy", "rb")
feats_s3_train = np.load(file_s3)
file_s3.close()

In [18]:
_, all_feats_test_norm_s1 = normalize_features(feats_s1_train, all_feats_test_s1, 7)
_, all_feats_test_norm_s2 = normalize_features(feats_s2_train, all_feats_test_s2, 7)
_, all_feats_test_norm_s3 = normalize_features(feats_s3_train, all_feats_test_s3, 7)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


## Dropping some of the features

We found that the lower-frequency biorhythm bandpowers did not provide much information, with almost all of the values being 0, so we dropped them after the extraction process.

In [19]:
# Problematic features, drop them for now. 
def clean_features(feats):
    bad_feat_inds = np.concatenate((np.arange(2,len(feats.transpose()),7),
                                   np.arange(3,len(feats.transpose()),7),
                                   np.arange(4,len(feats.transpose()),7),
                                  ))
    feats_cleaned = np.delete(feats, bad_feat_inds, axis=1)
    
    return feats_cleaned

all_feats_test_norm_s1 = clean_features(all_feats_test_norm_s1)
all_feats_test_norm_s2 = clean_features(all_feats_test_norm_s2)
all_feats_test_norm_s3 = clean_features(all_feats_test_norm_s3)

## Prediction using trained models

There are 3 individual models for each subject that have been trained.

In [20]:
reg_rfr_s1 = pickle.load(open('subject1_rfr_1000_run2.model', 'rb'))
pred_s1 = reg_rfr_s1.predict(all_feats_test_norm_s1)

reg_rfr_s2 = pickle.load(open('subject2_rfr_1000_run2.model', 'rb'))
pred_s2 = reg_rfr_s2.predict(all_feats_test_norm_s2)

reg_rfr_s3 = pickle.load(open('subject3_rfr_1000_run2.model', 'rb'))
pred_s3 = reg_rfr_s3.predict(all_feats_test_norm_s3)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid resu

## Post-processing the predictions

We first upsampled the predictions using a cubic spline, and then gaussian filtered the outputs to remove noise from the predictions.

In [22]:
def spline_preds(preds, time_length):
  # N samples
  preds_sample_orig = np.arange(len(preds))
  
  # T time points
  preds_sample_target = np.linspace(0,len(preds),time_length)
  preds = preds.transpose()

  preds_interp = []
  
  for finger_preds in preds:
      f = CubicSpline(preds_sample_orig, finger_preds, bc_type="natural")
      new_preds = f(preds_sample_target)
      preds_interp.append(new_preds)
  
  preds_interp = np.array(preds_interp).transpose()
  
  return preds_interp


def convolve_gaussian(preds):
  preds_t = preds.transpose()
  preds_t_convolve = []
  
  fs = 1000
  gaussian_filter = np.exp(-1*(np.arange(int(-1*1000),int(1*1000)))**2/(0.75*1000)**2)
  gaussian_filter_scaled = 1/np.sum(gaussian_filter) * gaussian_filter
  
  for row in preds_t:
      preds_t_convolve.append(np.convolve(gaussian_filter_scaled, row, "same"))
  
  return np.array(preds_t_convolve).transpose()
    

pred1_test_postprocess = convolve_gaussian(spline_preds(pred_s1, len(test_data_s1)))
pred2_test_postprocess = convolve_gaussian(spline_preds(pred_s2, len(test_data_s2)))
pred3_test_postprocess = convolve_gaussian(spline_preds(pred_s3, len(test_data_s3)))

print(np.shape(pred1_test_postprocess))
print(np.shape(pred2_test_postprocess))
print(np.shape(pred3_test_postprocess))

(147500, 5)
(147500, 5)
(147500, 5)


In [32]:
predictions_array = np.zeros((3,1), dtype=object)
predictions_array[0,0] = pred1_test_postprocess
predictions_array[1,0] = pred2_test_postprocess
predictions_array[2,0] = pred3_test_postprocess

savemat('predictions.mat', {'predicted_dg':predictions_array})

print("Predictions saved.")

Predictions saved.
